In [1]:
import tensorflow as tf
import numpy as np

import tensorflow_datasets as tfds

C:\Users\riyab\anaconda3\envs\py3-TF2.0\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mnist_dataset,mnist_info=tfds.load(name="mnist",with_info=True,as_supervised=True)

In [3]:
mnist_train=mnist_dataset["train"]
mnist_test=mnist_dataset["test"]

num_validation_sample=0.1*mnist_info.splits["train"].num_examples
num_validation_sample=tf.cast(num_validation_sample,tf.int64)

num_test_sample=mnist_info.splits["test"].num_examples
num_test_sample=tf.cast(num_test_sample,tf.int64)

def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)

BUFFER_SIZE=10000

shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validation_sample)
train_data=shuffled_train_and_validation_data.skip(num_validation_sample)


BATCH_SIZE=150

train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_sample)
test_data=test_data.batch(num_test_sample)

validation_inputs,validation_targets=next(iter(validation_data))

Outlining the Model

In [4]:
from tensorflow import keras

In [5]:
input_size = 784
output_size = 10

hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    
    
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 3rd hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 4th hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 5th hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [6]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics="accuracy")

Train

In [7]:
NUM_EPOCHS=5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
360/360 - 15s - loss: 0.2388 - accuracy: 0.9281 - val_loss: 0.1775 - val_accuracy: 0.9445
Epoch 2/5
360/360 - 13s - loss: 0.1031 - accuracy: 0.9685 - val_loss: 0.0778 - val_accuracy: 0.9757
Epoch 3/5
360/360 - 17s - loss: 0.0691 - accuracy: 0.9788 - val_loss: 0.0661 - val_accuracy: 0.9802
Epoch 4/5
360/360 - 17s - loss: 0.0540 - accuracy: 0.9839 - val_loss: 0.0712 - val_accuracy: 0.9790
Epoch 5/5
360/360 - 14s - loss: 0.0442 - accuracy: 0.9868 - val_loss: 0.0736 - val_accuracy: 0.9798


In [34]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 0s 997us/step - loss: 0.1023 - accuracy: 0.9746
